In [58]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
import sklearn
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import imblearn
from imblearn.combine import SMOTEENN 
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score
from numpy import mean
from numpy import std

## Data Collection

In [26]:
#Import hate-speech annotated database
df = pd.read_csv('labeled_data.csv',header=None)

#Import Boys and Girls Chatting Dataset - Non-offensive Words 



## Data Cleaning

In [27]:
#Clean Data and Classes

newclass = []
for i in df[0]:
    if i == 0 or i == 1:
        newclass.append(1)
    if i == 2:
        newclass.append(2)
df[0] = newclass
#There is now no "hate speech" class and text is deemed as either offensive or inoffensive




In [28]:
#we changed the tweets into lower case. Also,
#we removed all the URLs, usernames, white spaces, hashtags,
#punctuations and stop-words using pattern matching
#techniques from the collected tweets
#We will use RE to pattern match to just return the tweets 
import re
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem:re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem).strip())
    df[text_field] = df[text_field].apply(lambda elem: ' '.join(elem.split()))
    return df
for i in range(2):
    df = clean_text(df,1)





In [30]:
#Remove 10000 Values from df as it is imbalanced
#Want to randomly cut out 10000 entries for undersampling
print(df[0].value_counts())

array_list = list(range(0, len(df)))
random.shuffle(array_list)
count = 0
for i in array_list:
    if df[0][i] == 1:
        df = df.drop(i)
        count+=1
    if count == 10000:
        break
        
print(df[0].value_counts())
#Want to randomly add around 4000 entries 

1    20620
2     4163
Name: 0, dtype: int64
1    10620
2     4163
Name: 0, dtype: int64


## Feature Extraction

In [32]:
#Feature Extraction / Pipeline
pipeline_svc = Pipeline([
    ('tfidfT',  TfidfTransformer()),
    ('svc', SVC())])

## Data Splitting, Testing and Validation

In [59]:
#cross validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer()),
                           ('model', SVC(C=10, 
                                         gamma=0.1, 
                                         kernel='sigmoid', 
                                         random_state=42))])
cv = KFold(n_splits=10, random_state=None)

acc_score = []
scores = cross_val_score(pipeline, df[1],df[0], scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))


Accuracy: 0.942 (0.012)


In [60]:
X_train, X_test, y_train, y_test = train_test_split(df[1],df[0])
model = pipeline.fit(X_train, y_train)
y_predict = model.predict(X_test)
f1_score(y_test, y_predict)

0.961808854265628

In [78]:
print(model.predict(["Hi There"]))

[2]
